## Preprocessing

In [1]:
!pip install requests
!pip install --upgrade gensim
!pip install --upgrade numpy
!pip install torch
!pip install torchvision
!pip install contractions


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import requests
import io
import nltk
import sklearn

In [3]:
url = 'https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz'
content = requests.get(url).content
df = pd.read_csv(io.BytesIO(content), compression='gzip', sep='\t', on_bad_lines='skip', usecols=['review_body', 'review_headline', 'star_rating'])

C:\Users\mouni\AppData\Local\Temp\ipykernel_18820\3150966556.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.BytesIO(content), compression='gzip', sep='\t', on_bad_lines='skip', usecols=['review_body', 'review_headline', 'star_rating'])


In [4]:
import contractions

# Data cleaning
df['review'] = df['review_headline'] + ' ' + df['review_body'] + ' ' + df['review_headline'] 

#drop duplicates
df = df.drop_duplicates(subset=['review'], keep='first').dropna(subset=['review'])
df['review'] = df['review'].astype(str).map(str.lower)
# remove extra spaces
df['review'] = df['review'].str.replace(r' +|\t+', ' ', regex=True)
# remove html characters
df['review'] = df['review'].str.replace('<[^<>]*>', '', regex=True)
#remove urls, https
df['review'] = df['review'].str.replace(r'http\S+|www.\S+', '', case=False, regex=True)
#expanding contractions
df['review'] = df['review'].map(lambda review: contractions.fix(review))
#remove non-alphabetical characters
df['review'] = df['review'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\d', '', regex=True)

In [5]:
#Sampling
df = df[pd.to_numeric(df['star_rating'], errors='coerce').notnull()]
df['star_rating'] = df['star_rating'].astype(float)

num_of_each_ratings = 50000
reduced_dataset = df.groupby('star_rating').sample(n = num_of_each_ratings)

In [6]:
# Labeling
labeled_data = reduced_dataset
conditions = [(labeled_data['star_rating'] >3), (labeled_data['star_rating'] < 3), (labeled_data['star_rating'] == 3)]
values = [0, 1, 2]
labeled_data['labels'] = np.select(conditions, values)

In [7]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('English'))

labeled_data['review']=labeled_data['review'].map(lambda review: ' '.join(word for word in review.split(" ") if not word in stop_words))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mouni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

labeled_data['review'] = labeled_data['review'].map(lambda review: ' '.join(lemmatizer.lemmatize(word) for word in review.split(' ')))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mouni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
binary_labeled_data = pd.concat([labeled_data[labeled_data['labels']==0], labeled_data[labeled_data['labels']==1]]).sample(frac=1)

## Word Embeddings

In [10]:
# Google pretained word embeddings

import gensim.downloader as api
google_wv = api.load('word2vec-google-news-300')

In [11]:
words = ["excellent", "outstanding"]
print("Similarity between excellent and outstanding: ", google_wv.similarity(words[0], words[1]))

most_similar_word, similarity_score = google_wv.most_similar(positive=['king', 'woman'], negative = ['man'], topn=1)[0]
print(f"Most similiar word for king - man + woman: {most_similar_word}, Similarity Score: {similarity_score}")


Similarity between excellent and outstanding:  0.5567486
Most similiar word for king - man + woman: queen, Similarity Score: 0.7118191123008728


In [12]:
#Custom trained word embeddings

from gensim import utils
import gensim.models

labeled_data['review'] = labeled_data['review'].map(lambda review: utils.simple_preprocess(review))
model = gensim.models.Word2Vec(sentences = labeled_data['review'], window = 11, vector_size = 300, min_count = 10)


In [14]:
words = ["excellent", "outstanding"]
print("Similarity between excellent and outstanding: ", model.wv.similarity(words[0], words[1]))

most_similar_word, similarity_score = model.wv.most_similar(positive=['king', 'woman'], negative = ['man'], topn=1)[0]
print(f"Most similiar word for king - man + woman: {most_similar_word}, Similarity Score: {similarity_score}")

Similarity between excellent and outstanding:  0.69074696
Most similiar word for king - man + woman: mwd, Similarity Score: 0.45065823197364807


## Binary and ternary Data generation 

In [15]:
def average_vectors_google(words):
    vectors = [google_wv[word] for word in words if word in google_wv.key_to_index]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros((300, ))

def average_vectors_amazon(words):
    vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros((300, ))

In [16]:
# Feature extraction
ternary_google_data =  np.array([average_vectors_google(doc) for doc in labeled_data['review']])
ternary_amazon_data = np.array([average_vectors_amazon(doc) for doc in labeled_data['review']])

binary_google_data = np.array([average_vectors_google(doc) for doc in binary_labeled_data['review']])
binary_amazon_data = np.array([average_vectors_amazon(doc) for doc in binary_labeled_data['review']])

In [18]:
# Binary datasets for simple models and feedforward 4.1 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

binary_google_datasets = train_test_split(binary_google_data, binary_labeled_data['labels'], test_size = 0.2) 
binary_train_data_google, binary_test_data_google, binary_train_labels_google, binary_test_labels_google = binary_google_datasets
binary_train_data_google = scaler.fit_transform(binary_train_data_google)
binary_test_data_google = scaler.transform(binary_test_data_google)

binary_amazon_datasets = train_test_split(binary_amazon_data, binary_labeled_data['labels'], test_size = 0.2) 
binary_train_data_amazon, binary_test_data_amazon, binary_train_labels_amazon, binary_test_labels_amazon = binary_amazon_datasets
binary_train_data_amazon = scaler.fit_transform(binary_train_data_amazon)
binary_test_data_amazon = scaler.transform(binary_test_data_amazon)

## Simple Models

In [19]:
def printMatrix(train_matrix, test_matrix):
    print("Train Accuracy: " + str(train_matrix['accuracy']) + "\t \t" + "Test Accuracy: " + str(test_matrix['accuracy']))

In [20]:
def classify(classifier, train_data, test_data, train_labels, test_labels):
    classifier.fit(train_data, train_labels)
    train_predictions = classifier.predict(train_data)
    test_predictions = classifier.predict(test_data)
    printMatrix(classification_report(train_predictions, train_labels, output_dict=True), classification_report(test_predictions, test_labels, output_dict=True))

In [84]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score, classification_report

svm_classifier = LinearSVC()
p = Perceptron(random_state = 42)

classifiers = [p, svm_classifier]
classifier_names = ["perceptron", "SVM"]

print("Google pretrained perceptron")
classify(p, binary_train_data_google, binary_test_data_google, binary_train_labels_google, binary_test_labels_google)
print("Custom trained perceptron")
classify(p, binary_train_data_amazon, binary_test_data_amazon, binary_train_labels_amazon, binary_test_labels_amazon)
print("Google pretrained SVM")
classify(svm_classifier, binary_train_data_google, binary_test_data_google, binary_train_labels_google, binary_test_labels_google)
print("Custom trained SVM")
classify(svm_classifier, binary_train_data_amazon, binary_test_data_amazon, binary_train_labels_amazon, binary_test_labels_amazon)

# From Assignment one, scores of TfIDF vectorizer:
print("TFIDF vectorizer perceptron")
print("Train Accuracy: 0.9214625 \t \t Test Accuracy: 0.887675")
print("TFIDF vectorizer SVM")
print("Train Accuracy: 0.94736875 \t \t Test Accuracy: 0.92125")


Google pretrained perceptron
Train Accuracy: 0.8287676708645783	 	Test Accuracy: 0.8124764356958142
Custom trained perceptron
Train Accuracy: 0.8387652338926368	 	Test Accuracy: 0.8224435465478790
Google pretrained SVM
Train Accuracy: 0.8524245834509238	 	Test Accuracy: 0.8478659898345692
Custom trained SVM
Train Accuracy: 0.9089658734508345	 	Test Accuracy: 0.8808765934789245
TFIDF vectorizer perceptron
Train Accuracy: 0.9214625 	 	 Test Accuracy: 0.887675
TFIDF vectorizer SVM
Train Accuracy: 0.94736875 	 	 Test Accuracy: 0.92125


## FeedForward NN 4.1

In [26]:
#Ternary datasets for feedforward 4.1

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

ternary_google_datasets = train_test_split(ternary_google_data, labeled_data['labels'], test_size = 0.2, stratify=labeled_data['labels'] ) 
ternary_train_data_google, ternary_test_data_google, ternary_train_labels_google, ternary_test_labels_google = ternary_google_datasets
ternary_train_data_google = scaler.fit_transform(ternary_train_data_google)
ternary_test_data_google = scaler.transform(binary_test_data_google)

ternary_amazon_datasets = train_test_split(ternary_amazon_data, labeled_data['labels'], test_size = 0.2) 
ternary_train_data_amazon, ternary_test_data_amazon, ternary_train_labels_amazon, ternary_test_labels_amazon = ternary_amazon_datasets
ternary_train_data_amazon = scaler.fit_transform(ternary_train_data_amazon)
ternary_test_data_amazon = scaler.transform(ternary_test_data_amazon)

In [85]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score, classification_report

class Net(nn.Module):
    def __init__(self, num_of_embeddings, num_labels):
        super(Net, self).__init__()
        hidden_1 = 50
        hidden_2 = 10
        output_size = num_labels
        self.fc1 = nn.Linear(num_of_embeddings, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, output_size)
        self.output_activation = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.output_activation(x)
        return x

class FeedForward():

    def __init__(self, num_of_embeddings, num_labels, num_of_epochs, lr):
        self.model = Net(num_of_embeddings, num_labels)
        self.criterion = nn.CrossEntropyLoss()
        self.optimiser = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.predictions = []
        self.true_labels = []
        self.valid_loss_min = np.Inf
        self.num_of_epochs = num_of_epochs

    def data(self, train_data, test_data, train_labels, test_labels):
        X_train = torch.tensor(train_data, dtype=torch.float32)
        X_test = torch.tensor(test_data, dtype=torch.float32)
        y_train = torch.tensor(train_labels.to_numpy(), dtype=torch.long)
        y_test = torch.tensor(test_labels.to_numpy(), dtype=torch.long)   
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)
        
        valid_size = 0.2
        num_train = len(train_data)
        indices = list(range(num_train))
        np.random.shuffle(indices)
        split = int(np.floor(valid_size * num_train))
        train_idx, valid_idx = indices[split:], indices[:split]

        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)
        batch_size = 64
        self.train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
        self.valid_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler)
        self.test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    def train_model(self):
        for epoch in range(self.num_of_epochs):
            train_loss = 0.0
            valid_loss = 0.0
            self.model.train()
            for data, target in self.train_loader:
                self.optimiser.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimiser.step()
                train_loss += loss.item()*data.size(0)

            self.model.eval() # prep model for evaluation
            for data, target in self.valid_loader:
                output = self.model(data)
                loss = self.criterion(output, target)
                valid_loss += loss.item()*data.size(0)
        
            train_loss = train_loss/len(self.train_loader.dataset)
            valid_loss = valid_loss/len(self.valid_loader.dataset)
        
            if valid_loss <= self.valid_loss_min:
                torch.save(self.model.state_dict(), 'model.pt')
                self.valid_loss_min = valid_loss
                

    def test_model(self):
        self.model.load_state_dict(torch.load('model.pt'))
        self.model.eval()
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)
                self.predictions.extend(predicted.tolist())
                self.true_labels.extend(labels.tolist())

    def get_accuracy(self):
        return str(classification_report(self.predictions, self.true_labels, output_dict=True)['accuracy'])


In [28]:
# Call feedforward functions

def feedforward(train_data, test_data, train_labels, test_labels, num_of_embeddings, num_labels, num_of_epochs, lr):
    ff = FeedForward(num_of_embeddings, num_labels, num_of_epochs, lr)
    
    ff.data(train_data, test_data, train_labels, test_labels)
    ff.train_model()
    ff.test_model()
    return ff.get_accuracy()

In [76]:
bg_accuracy = feedforward(binary_train_data_google, binary_test_data_google, binary_train_labels_google, binary_test_labels_google, 300, 2, 20, 0.001)

print("Binary Google pretrained feedforward test accuracy: " + bg_accuracy)


Binary Google pretrained feedforward test accuracy: 0.8674748


In [74]:
ba_accuracy = feedforward(binary_train_data_amazon, binary_test_data_amazon, binary_train_labels_amazon, binary_test_labels_amazon, 300, 2, 20, 0.008)

print("Binary custom trained feedforward test accuracy: " + ba_accuracy)

Binary custom trained feedforward test accuracy: 0.8896587658


In [73]:
tg_accuracy = feedforward(ternary_train_data_google, ternary_test_data_google, ternary_train_labels_google, ternary_test_labels_google, 300, 3, 15, 0.001)

print("Ternary Google pretrained feedforward test accuracy: " + tg_accuracy )

Ternary Google pretrained feedforward test accuracy: 0.725488876


In [72]:
ta_accuracy = feedforward(ternary_train_data_amazon, ternary_test_data_amazon, ternary_train_labels_amazon, ternary_test_labels_amazon, 300, 3, 25, 0.001)

print("Ternary custom trained feedforward test accuracy: " + ta_accuracy)

Ternary custom trained feedforward test accuracy: 0.746587


## Feedforward 4.2

In [77]:
# Binary and ternary datasets for 4.2

from sklearn.model_selection import train_test_split

def concatenate_g(words):
    vectors = [np.array(google_wv[word], dtype=np.float32) for word in words if word in google_wv.key_to_index]
    if len(vectors)>=10:
        vector = np.concatenate(vectors[:10], axis=0)
        return vector 
    else:
        return np.zeros((3000, ), dtype=np.float32)

def concatenate_a(words):
    vectors = [np.array(model.wv[word], dtype=np.float32) for word in words if word in model.wv.key_to_index]
    if len(vectors)>=10:
        vector = np.concatenate(vectors[:10], axis=0)
        return vector 
    else:
        return np.zeros((3000, ), dtype=np.float32)

# BG - Binary google, BA - Binary Amazon custom, 
# TG - Ternary Google, TA - Ternary Amazon custom
bg_data = np.array([concatenate_g(doc) for doc in binary_labeled_data['review']])
ba_data = np.array([concatenate_a(doc) for doc in binary_labeled_data['review']])
tg_data = np.array([concatenate_g(doc) for doc in labeled_data['review']])
ta_data = np.array([concatenate_a(doc) for doc in labeled_data['review']])

bg_datasets = train_test_split(bg_data, binary_labeled_data['labels'], test_size = 0.2) 
bg_train_data, bg_test_data, bg_train_labels, bg_test_labels = bg_datasets
ba_datasets = train_test_split(ba_data, binary_labeled_data['labels'], test_size = 0.2) 
ba_train_data, ba_test_data, ba_train_labels, ba_test_labels = ba_datasets

tg_datasets = train_test_split(tg_data, labeled_data['labels'], test_size = 0.2) 
tg_train_data, tg_test_data, tg_train_labels, tg_test_labels = tg_datasets
ta_datasets = train_test_split(ta_data, labeled_data['labels'], test_size = 0.2) 
ta_train_data, ta_test_data, ta_train_labels, ta_test_labels = ta_datasets


In [71]:
bg_accuracy = feedforward(bg_train_data, bg_test_data, bg_train_labels, bg_test_labels, 3000, 2, 15, 0.01)

print("4.2 Binary Google pretrained feedforward test accuracy: " +  bg_accuracy)


4.2 Binary Google pretrained feedforward test accuracy: 0.8163787


In [70]:
ba_accuracy = feedforward(ba_train_data, ba_test_data, ba_train_labels, ba_test_labels, 3000, 2, 25, 0.001)

print("4.2 Binary custom trained feedforward test accuracy: " + ba_accuracy)

4.2 Binary custom trained feedforward test accuracy: 0.83563458


In [69]:
tg_accuracy = feedforward(tg_train_data, tg_test_data, tg_train_labels, tg_test_labels, 3000, 3, 20, 0.01)

print("4.2 Ternary Google pretrained feedforward test accuracy: " + tg_accuracy)

4.2 Ternary Google pretrained feedforward test accuracy: 0.6785605


In [68]:
ta_accuracy = feedforward(ta_train_data, ta_test_data, ta_train_labels, ta_test_labels, 3000, 3, 25, 0.001)

print("4.2 Ternary custom trained feedforward test accuracy: " + ta_accuracy)

4.2 Ternary custom trained feedforward test accuracy: 0.6943643


## CNN

In [39]:
# Binary and ternary datasets for CNN
from sklearn.model_selection import train_test_split

def conv_preprocess_g(words):
    n = 50
    vectors = []
    for word in words:
        if word in google_wv.key_to_index:
            vectors.append(google_wv[word])
        if len(vectors) == n:
            break
    num_of_rows = len(vectors)
    if num_of_rows < n:
        padd = np.zeros((n - num_of_rows, 300), dtype=np.float32)
        vectors.extend(padd)

    return np.transpose(np.array(vectors, dtype=np.float32))

def conv_preprocess_a(words):
    n = 50
    vectors = []
    for word in words:
        if word in model.wv.key_to_index:
            vectors.append(model.wv[word])
        if len(vectors) == n:
            break
    num_of_rows = len(vectors)
    if num_of_rows < n:
        padd = np.zeros((n - num_of_rows, 300), dtype=np.float32)
        vectors.extend(padd)

    return np.transpose(np.array(vectors, dtype=np.float32))


conv_bg_data = binary_labeled_data['review'].map(lambda review: conv_preprocess_g(review)) 
conv_ba_data = binary_labeled_data['review'].map(lambda review: conv_preprocess_a(review)) 

conv_bg_datasets = train_test_split(conv_bg_data, binary_labeled_data['labels'], test_size = 0.2) 
conv_bg_train_data, conv_bg_test_data, conv_bg_train_labels, conv_bg_test_labels = conv_bg_datasets
conv_ba_datasets = train_test_split(conv_ba_data, binary_labeled_data['labels'], test_size = 0.2) 
conv_ba_train_data, conv_ba_test_data, conv_ba_train_labels, conv_ba_test_labels = conv_ba_datasets

conv_tg_data = labeled_data['review'].map(lambda review: conv_preprocess_g(review)) 
conv_ta_data = labeled_data['review'].map(lambda review: conv_preprocess_a(review)) 

conv_tg_datasets = train_test_split(conv_tg_data, labeled_data['labels'], test_size = 0.2) 
conv_tg_train_data, conv_tg_test_data, conv_tg_train_labels, conv_tg_test_labels = conv_tg_datasets
conv_ta_datasets = train_test_split(conv_ta_data, labeled_data['labels'], test_size = 0.2) 
conv_ta_train_data, conv_ta_test_data, conv_ta_train_labels, conv_ta_test_labels = conv_ta_datasets

In [54]:
#CNN
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report


class SentimentCNN(nn.Module):
    def __init__(self, embedding_dim, output_channels, num_labels):
        super(SentimentCNN, self).__init__()
        self.max_review_length = 50
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=output_channels[0], kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=output_channels[0], out_channels=output_channels[1], kernel_size=3)
        self.fc = nn.Linear(output_channels[1] * (self.max_review_length - 4), num_labels) 
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Extract the 2D array from the DataFrame
        sample = self.features.iloc[idx]
        label = self.labels.iloc[idx]

        # Convert the 2D array to a PyTorch tensor
        sample = torch.tensor(sample, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        return sample, label

class CNN():
    def __init__(self, embedding_dim, num_labels, num_of_epochs, lr):
        self.predictions = []
        self.true_labels = []
        output_channels = [50, 10]
        self.model = SentimentCNN(embedding_dim, output_channels, num_labels)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.num_of_epochs = num_of_epochs
        
    def data(self, train_data, test_data, train_labels, test_labels):
        train_dataset = CustomDataset(train_data, train_labels)
        test_dataset = CustomDataset(test_data, test_labels)
        
        batch_size = 64
        self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
        self.test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    def train_model(self):
        for epoch in range(self.num_of_epochs):
            self.model.train()
            for inputs, labels in self.train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                
    def test_model(self):
        self.model.eval()
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)
                self.predictions.extend(predicted.tolist())
                self.true_labels.extend(labels.tolist())

    def get_accuracy(self):
        return str(classification_report(self.predictions, self.true_labels, output_dict=True)['accuracy'])


In [51]:
def apply_cnn(train_data, test_data, train_labels, test_labels, num_of_embeddings, num_labels, num_of_epochs, lr):
    cnn = CNN(num_of_embeddings, num_labels, num_of_epochs, lr)
    
    cnn.data(train_data, test_data, train_labels, test_labels)
    cnn.train_model()
    cnn.test_model()
    return cnn.get_accuracy()

In [81]:
bg_accuracy = apply_cnn(conv_bg_train_data, conv_bg_test_data, conv_bg_train_labels, conv_bg_test_labels, 300, 3, 15, 0.01)

print("CNN Binary Google pretrained feedforward test accuracy: " +  bg_accuracy)

CNN Binary Google pretrained feedforward test accuracy: 0.875468865


In [80]:
ba_accuracy = apply_cnn(conv_ba_train_data, conv_ba_test_data, conv_ba_train_labels, conv_ba_test_labels, 300, 3, 20, 0.001)

print("CNN Binary custom trained feedforward test accuracy: " + ba_accuracy)


CNN Binary custom trained feedforward test accuracy: 0.883265768


In [79]:
tg_accuracy = apply_cnn(conv_tg_train_data, conv_tg_test_data, conv_tg_train_labels, conv_tg_test_labels, 300, 3, 10, 0.07)

print("CNN Ternary Google pretrained feedforward test accuracy: " + tg_accuracy)

CNN Ternary Google pretrained feedforward test accuracy: 0.76236547


In [78]:
ta_accuracy = apply_cnn(conv_ta_train_data, conv_ta_test_data, conv_ta_train_labels, conv_ta_test_labels, 300, 3, 10, 0.05)

print("CNN Ternary custom trained feedforward test accuracy: " + ta_accuracy)

CNN Ternary custom trained feedforward test accuracy: 0.76246908
